# VPN basics

## Introduction to Modern WAN Technology

* Traditional router based network used Leased line connections 

| __Advantage__ | __Disadvantage__ |
|---|---|
| Secure | Expensive |
| High Throughput | Permenent physical connections |
| Superior Quality and Reliable | Not-Scalable |

* During 1990s VPNs were introduced that emulates a point-to-point network over a WAN. 
* Customers opted for VPN services for reducing their OpEx and to bring flexibility. 
* Examples : X.25, Frame-Relay, ATM, GRE, DMVPN, IPSec, MPLS, L2TPv3

![](mpls_figs/vpn_basic.png)

* Advantages of VPN 

| Features | Descriptions |
|---|---|
| __Cost effective__ | replaces the expensive leased line connections | 
| __Scalable__ | Adding a branch is simple by connecting the CE router to the nearest branch office |
| __Secure__ | Uses modern Encryption protocol such as IPSec with AES ans HMAC-SHA | 
| __Better Performance__ | Uses fiber based data-plane | 
| __Flexible__ | Does not rely on underlaying protocols | 
| __Reliable__ |  something... |
    
* MPLS Terms 

![](mpls_figs/vpn_terms.png)

## VPN Models

| __Overlay Model__ | __Peer-to-peer model__ |
|---|---|
| ISP provides virtual P2P links between customer-sites <br> __Layer 2__ : FR, ATM, x.25 <br> __Layer   3__ : GRE, DMVPN, IPSec, L2TPv3, SSL VPN | ISP participates in the customer routing <br> GET-VPN, MPLS |

* __ACL based Shared Routing__ : Each PE router was configured with ACL that would prevent any cross-talk betwenn customer routers (More OpEx)
* __Split Routing__ : Every customer was given a dedicated PE Routers (More CapEx)
* __MPLS VPN__ Solves the problem with __Virtual Routing an Forwarding (VRF)__ where the PE maintains virtual routing table for each customer, isolated from eachother, apart from the global routing table. 

|  | __Advantages__ | __Disadvantages__ |
| --- | --- |---|
| | Well known and easy to implement | Optimimal routing requires full-mesh of Virtual Circuits (VC) |
|__Overlay__  | ISP does not participate in cutomer routing | VC to be provissioned manually | 
| | Cusomer and ISP networks are well isolated | Incurs encapsulation overhead | 
|---|---|---|
| | Gurantees optimal routing between customers | ISP must apply filter to the Customer Links  |
|  | Easier to provision and additional VPN |  ISP is responsible for customer convergence  |
| __P2P__| Only Site provisioned, not links between them | PE routers carry all the routes for all the customer  | 
| | | Secure environment must be provided for customers  |
| | | Complex configuration |
| | | ISP needs detailed IP routing knowledge |

## MPLS VPNs

* Forwards packets based on labels instead of IP.
* Combines the best of both Overlay and P2P models. 
* The PE router maintans seperate VRF routing tables for each customer (Multi-Tenancy). 
* A VPN-V4 peering is eshtablished between PE-PE routers (Similar to a Tunnel). 
* The PE router receives a normal IP packet and adds a VPN-Label, the labeled packet will be forwarded to proper destination without seeing the content inside. 
* Core ISP routes does not maintain any customer routes : __BGP Free-Core__ 

## Cisco Express Forwarding (CEF) 

![](mpls_figs/mpls_cef.png)

* __Layer 3 switch processing__: When a packet appears to the ingress interface of a router, 
    * it sees the destination network IP
    * Performs a Routing table lookup for a next-hop IP address
    * finds the exit interface of the next-hop interface 
    * Creates a L2 header, update TTL on IP Header 
    * Assemble and send it to the egreee interface 
* There are Three Switching modes

| Mode | Dription | 
|---|---|
| __Process Switching__ | Requires CPU to be involved with every forwarding decision|
| __Fast Switching__ | Uses CPU but also caches the most frequently translated informations |
| __CEF__ | Optimises the process by using dedicated hardware to store pre-computed data |

* __CEF in details__
    * L3 routing table (RIB) is downloaded into the hardware (FIB)
    * Lookup is done on the hardwire i.e. in wire-speed 
    * The proactive replication of RIB into FIB makes the process optimized. 
    
![](mpls_figs/mpls_cef_lookup.png)

* To verify CEF : `sh ip cef` maps prefix to nexthop and exit interface 
* To Disable CEF and move into fast-switching : `no ip cef` 

# MPLS Label Distibution

## MPLS introduction 

![](mpls_figs/mpls_lib.png)

* In traditional IP routing method, a destination prefix from a IP packet is checked on the Routing table for the next hop and the exit interface. Routing protocols are used to distribute the routing information 
* MPLS relies on the traditional IP routing. A customer packet comes to the ISP edge as a normal IP Packet. The ISP Edge __pushes__ a label to the packet while it enters the ISP network. Throught the ISP core the labels are __Swapped__ by core routers. The exit edge router __Pops__ the label and make it a normal IP packet again. It is Called __Multi-protocols__ as it runs on any L2 links (ATM, FR, PPP, HDLC etc.), the __Label Switching__ is provided by building Lable binding information (LIB, LFIB) in the hardware lavel using CEF. 
 
### Basic MPLS Terms 

* __Core LSR__ : Label switched routers are responsible for swapping the label 
* __Edge LSR__ : Pushes or Pops labels (Ingress LSR and Egress LSR)
* __LSP__ : Label switch Path is whre the packets are forwarded based on the labels (not based on IP)

### Benifits of MPLS 

* MPLS supports multicast routing 
* MPLS decreses the overhead on core-routers 
* BGP-Free Core : Core routers don't need to maintan any customer routes 
* Support Non-IP protocols : as the forwarding process doesn't sees the L3 headers 
* Useful for VPN, TE (Traffic Engineering) , QoS, AToM (Any Transport Over MPLS)

## MPLS Labels and Stacks 

![](mpls_figs/mpls_label.png)

* 32 bits Locally significant. 
  * 20 bits : Label 
  * 3 bits  : Experimental bits, used for QoS marking information
  * 1 bit   : Bottom of Stack (BoS) indicates if it is the last label or not.
  * 8 bits  : TTL field to prevent loops. 
* Labels are distributed by LDP. 

### MPLS Lable stack

* Usually one one label is assigned to a packet, but multipe lables in a label stack are supprted. 
* this is cenarios may produce more than one label.
    * __MPLS VPN (Two lables)__ : top label points to the egress router and the second label indentifies the VPN. 
    * __MPLS TE (Two or more lables)__: TE uses RSVP instead of LDP. The top label points to the endpoint of the traffic engineering tunnel and the second label points to the destination 
    * MPLS VPNS combined with MPLS TE. 
    

## Sharing of the Label Information

![](mpls_figs/label_share.png)

* MPLS does not share the label iformation automatically, it needs a distribution protocol using the IGP in the ISP core. The following protols are majorly used to do so.

| Protocols | Descriptons | 
|-|-|
| __TDP__ | Cisco Prop <br> Obsolete <br> TCP port 711 |
| __LDP__ | Standard <br> Default on cisco <br> UDP port 646 |
| __RSVP__ | used for MPLS TE Labels | 

## Configuring LDP in ISP Core 

![](mpls_figs/lab1.png)

### Base Config 

```
!r1
conf t
	int f0/0
		ip add 1.1.1.1 255.255.255.0
		no sh 
	int l0
		ip add 10.1.1.1 255.255.255.0
	router os 1
		pass def 
		net 0.0.0.0 0.0.0.0 a 0
		no pass f0/0
end 

!r2
conf t
	int f0/0
		ip add 1.1.1.2 255.255.255.0
		no sh 
	int f0/1
		ip add 2.2.2.2 255.255.255.0
		no sh
	int l0
		ip add 20.1.1.1 255.255.255.0
	router os 1
		pass def 
		net 0.0.0.0 0.0.0.0 a 0
		no pass f0/0
		no pass f0/1
end 

!r3
conf t
	int f0/0
		ip add 2.2.2.3 255.255.255.0
		no sh 
	int f0/1
		ip add 3.3.3.3 255.255.255.0
		no sh
	int l0
		ip add 30.1.1.1 255.255.255.0
	router os 1
		pass def 
		net 0.0.0.0 0.0.0.0 a 0
		no pass f0/0
		no pass f0/1
end 

!r1
conf t
	int f0/0
		ip add 3.3.3.4 255.255.255.0
		no sh 
	int l0
		ip add 40.1.1.1 255.255.255.0
	router os 1
		pass def 
		net 0.0.0.0 0.0.0.0 a 0
		no pass f0/0
end 
```

__Pre-Requisete__ : CEF must be runing on the routers to enable LDP 

```
R1#sh ip cef 20.1.1.0 255.255.255.0
20.1.1.0/24, version 23, epoch 0, cached adjacency 1.1.1.2
0 packets, 0 bytes
  via 1.1.1.2, FastEthernet0/0, 0 dependencies
    next hop 1.1.1.2, FastEthernet0/0
    valid cached adjacency
    
R1#sh ip cef 30.1.1.0 255.255.255.0 
30.1.1.0/24, version 24, epoch 0, cached adjacency 1.1.1.2
0 packets, 0 bytes
  via 1.1.1.2, FastEthernet0/0, 0 dependencies
    next hop 1.1.1.2, FastEthernet0/0
    valid cached adjacency
    
R1#sh ip cef 40.1.1.0 255.255.255.0
40.1.1.0/24, version 25, epoch 0, cached adjacency 1.1.1.2
0 packets, 0 bytes
  via 1.1.1.2, FastEthernet0/0, 0 dependencies
    next hop 1.1.1.2, FastEthernet0/0
    valid cached adjacency
```


### Activating LDP 

```
!r1
conf t
    mpls label protocol ldp  ! optional
    mpls label range 100 199
    mpls ldp router-id loop0 ! optional but RID must be reachable 
    
    int f0/0
        mpls ip    ! activate ldp on the interface 
end 

!verification
sh mpls ldp nei    ! shows ldp neighburs  
sh mpls ldp int    ! shows ldp enabled interfaces
sh mpls ldp binding A.B.C.D CIDR       ! shows local binding 
sh mpls forwarding-table               ! shows LFIB
trace 40.1.1.1     ! shows the LSP 
```

## LDP Forwarding process 

* IGP builds routing table (FIB using CEF)
* LSR assigns a local label for each route learnt 
* LSR Share the label with nighbours using LDP 
* Based on the collected information, LSR builds their LFIB. 
* Routers will local prefix uses label ID = 3 called Implecit Null. 
* __TIB__: Tag Information Base, __TSR__: Tag Switch Router 

The following figure depicts the LIB of the 4 LSRs with respect to 40.1.1.0/24 prefix using __`sh mpls ldp binfing 40.1.1.0 24`__ command. 

![](mpls_figs/lab2.png)

The following list presents the complete LFIB desciption of all the routers. Using __`sh mpls forwarding-table`__ command 

__LFIB of Router R1__

```
Local  Outgoing    Prefix            Bytes tag  Outgoing   Next Hop    
tag    tag or VC   or Tunnel Id      switched   interface              
100    Pop tag     2.2.2.0/24        0          Fa0/0      1.1.1.2      
101    200         3.3.3.0/24        0          Fa0/0      1.1.1.2      
102    Pop tag     20.1.1.0/24       0          Fa0/0      1.1.1.2      
103    201         40.1.1.0/24       0          Fa0/0      1.1.1.2      
104    203         30.1.1.0/24       0          Fa0/0      1.1.1.2   
```

__LFIB of Router R2__

```
Local  Outgoing    Prefix            Bytes tag  Outgoing   Next Hop    
tag    tag or VC   or Tunnel Id      switched   interface              
200    Pop tag     3.3.3.0/24        0          Fa0/1      2.2.2.3      
201    302         40.1.1.0/24       1200       Fa0/1      2.2.2.3      
202    Pop tag     10.1.1.0/24       0          Fa0/0      1.1.1.1      
203    Pop tag     30.1.1.0/24       0          Fa0/1      2.2.2.3   
```

__LFIB of Router R3__

```
Local  Outgoing    Prefix            Bytes tag  Outgoing   Next Hop    
tag    tag or VC   or Tunnel Id      switched   interface              
300    Pop tag     1.1.1.0/24        1302       Fa0/0      2.2.2.2      
301    Pop tag     20.1.1.0/24       0          Fa0/0      2.2.2.2      
302    Pop tag     40.1.1.0/24       1260       Fa0/1      3.3.3.4      
303    202         10.1.1.0/24       0          Fa0/0      2.2.2.2      
```

__LFIB of Router 4__

```
Local  Outgoing    Prefix            Bytes tag  Outgoing   Next Hop    
tag    tag or VC   or Tunnel Id      switched   interface              
400    300         1.1.1.0/24        0          Fa0/0      3.3.3.3      
401    Pop tag     2.2.2.0/24        0          Fa0/0      3.3.3.3      
402    301         20.1.1.0/24       0          Fa0/0      3.3.3.3      
403    303         10.1.1.0/24       0          Fa0/0      3.3.3.3      
404    Pop tag     30.1.1.0/24       0          Fa0/0      3.3.3.3   
```

## Penultimate Hop Popping (PHP)

![](mpls_figs/php.png)

* A built-in feature to __optimize__ the MPLS performance 
* PHP removes the requirement for a __double lookup__ to be performed on an egress PE. Without PHP, when a labeled packet appears on its egress LSR, the LSR first looks up the LFIB and finds the prefix is local. Then, it needs to lookup its FIB to find its exit interface. Therefore, two lookups. 
* Egress router assigns __imp-null__ (Label 3) to all its local prefixes and advertises to its LDP neighbours. The Penultimate (Second to Last) router pops the label and send the packet to the egress router as a normal IP packet. 
* The PHP router put a __pop-tag__ label at its LFIB for the prefixes it received __imp-null__. 
```
R3#sh mpls forwarding-table 40.1.1.0 24
Local  Outgoing    Prefix            Bytes tag  Outgoing   Next Hop    
tag    tag or VC   or Tunnel Id      switched   interface              
302    Pop tag     40.1.1.0/24       1260       Fa0/1      3.3.3.4 
```
* To disable PHP, use the __`mpls ldp explicit-null`__ command. The Egress router sends a __label=0__ instead of 3. It is recommanded when using __MPLS QoS__ , if the last router doesn't receive labeled packet then, the end-to-end Marking will not be guranteed. 

## LDP Troubleshoting 

### Possible issues 

* `mpls ip` command is not enabled in the interface 
* Protocol mismatch (LDP/TDP) global or at local interface level 
* Higher loopback IP is taken as RID which is not advertised by IGP
* Authentication mismatch 
* Port 646 is filterred 

# MPLS VPN 

## Introduction to MPLS VPN

![](mpls_figs/mpls_vpn.png)

* Packet forwarding takes place based on labels instead of the IP 
* Combines the best of both overlay and peer-to-peer model
* Customer advertises its routes to the PE router. 
* PE maintains customer routes into seperate routing table 
* a VPN-v4 peering must be established between PE-PE (e.g. a GRE Tunnel) 
* When Customer routes enters the ISP core, The PE __Pushes__ a label on the packet.
* ISP core routers (P) only swaps the label and forwards the labels based on the labels only. As a result the P routes don't need to maintain any customer routes, hence achiecing a __BGP Free Core__. 

## Steps to Configure MPLS L3 VPN

* __Step 1__: Configure IGP inside ISP core (mostly OSPF or IS-IS protocols are preferred)
* __Step 2__: Configure MPLS LDP inside the SP Core 
* __Step 3__: Create VRF, and assign RD, RT
* __Step 4__: Configure VPNv4 peering between both PE routers 
* __Step 5__: Configure Routing between PE and CE (Static/Default, IGP, BGP)
* __Step 6__: Configure Redistribution on PE Routers

## Virtual Routing and Forwarding (VRF)

![](mpls_figs/vrf.png)

* Provides a way to configure multiple virtual routing instances on a single physical router. 
* Keeps customer traffic and routing sepatate utilizing hardware resources parallelly 
* Without VRF one has to use Spit-Routing or Shared-Routing (ACL) methods. 
* The PE Router maintains other routes under its global routing table which is also isolated from the VRFs. 
* The primary reason of using a VRF is to prevent route-leaking within customers if multiple customer uses identical subnets. 
* Every VRF builds its won CEF table to maintan corresponding routes sepately. 

### Route Distingusher (RD)

![](mpls_figs/vrf_rd.png)

Route Distinguisher (RD) makes customer values unique in the ISP Core as multiple customer might use identical subnets and VRFs are locally significant.  

* RD value is optional for VRF-Lite (i.e. VRF without MPLS) but mandatory for VRF (i.e. with MPLS).
* RD value is a __64 bits__ prefix used to convert a client's non-unique 32bit IPv4 address into a globally unique __96 bits VPNv4__ (64+32) address to enable transport between PE-PE routers. 
* RD value is locally significant to a router 
* A VRF is not operational untill RD is defined 
* Common format : __ASN:nn__ or __ABCD:nn__ for RD
* Each VRF in a PE must have an unique RD value. 

### Route-Target (RT)

![](mpls_figs/vrf_rt.png)

Both RD and RT values are expressed in a same 64bit format, however they serve different purpose. Assume a scenario where all of your customer must get access to a central service (Database, Web-gateway etc.) or inter-customer routers are allowed due to company merger. since RD values are locally significant only, it has no control how to install routes on other VRFs and Thus, RD does not meet the requiement. RT values are extended BGP community attributes, therefore it can be sent over the VPNv4 link using the __export__ commnad by the MP-BGP, to let the receiving touter aware of the RT value. The receiving router may define in the VRF definition using the __import__ command, which routes to be installed. 

* A __64 bits BGP extended community__ that is attached to a VPNv4 BGP route to indicate its VPN membership.
* Any member of RT can be attached to a single route. 
* __Export RTs__
    * Indentifies the VPN membership to which the assocoated VRF belongs to. 
    * Attches RT values to a client route, when it it converted to a VPNv4 route.
* __Import RTs__
    * Selects which VPNv4 routes are to be installed into which VRF tables.
    * On the receiving PE router, a route is imported into a VRF only if at least one RT attached to the route matches at least one import RT configured in that VRF. 

## VRF Configuration

###  Lab Topology 

![](mpls_figs/lab3.png)

### Config Script 

__For 3600/3700 series routers__

```
conf t
    ip vrf VRF_NAME                  ! creates a VRF 
        rd RD_VALUE                  ! set RD value (local)  
        route-target import RT_VALUE ! RT value to install   
        route-target export RT_VALUE ! RT value to send over MP-BGP
    int IFACE
        ip vrf forwarding VRF_NAME   ! assign customer interface to VRF 
        ip address IP MASK           ! vrf allocation will remove all L3 info    
        no sh                        
end 
```

__For 7000/XR series routers__

```
conf t
    vrf definition VRF_NAME
        rd 500:1 
        address-family ipv4 | ipv6
            route-target export 500:1
            route-target export 500:1
end 
```

## VPNv4 Routing 

### Topology

![](mpls_figs/lab4.png)